In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [10]:
# Function to extract Product Title
def get_title(soup):

    try:
        title = soup.find("span", attrs={'id':'productTitle'}).text.strip()

    except AttributeError:
        title = ""

    return title

# Function to extract Product Price
def get_price(soup):

    try:
        price = (soup.find("span", attrs={'class':'a-offscreen'}).text)[1:].split('.')[0].replace(',','')

    except AttributeError:
        price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("span", attrs={'class':'a-icon-alt'}).text.strip().split(' ')[0]
        
        if rating == 'Previous':
            rating = "NA"
            
    except AttributeError:
        rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).text.split(' ')[0]

    except AttributeError:
        review_count = 0

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'}).find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [11]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=playstation+5&sprefix=playsta%2Caps%2C206&ref=nb_sb_ss_ts-doa-p_1_7"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [12]:
amazon_df

,title,price,rating,reviews,availability
0,Hogwarts Legacy | Standard Edition| PlayStatio...,4299,NA,0,"This item will be released on February 10, 2023."
1,God Of War Ragnarok | Launch Edition | PS5 Gam...,4799,4.7,163,In stock.
2,Uncharted: Legacy of Thieves Collection | PS5 ...,1329,4.3,163,In stock.
3,Dead Space | Standard Edition | PlayStation 5,4499,NA,0,"This item will be released on January 27, 2023."
4,PS5 God of War: Ragnarok Collector's Edition,13999,4.0,10,In stock.
5,PS5 Grand Theft Auto V,1999,4.4,109,In stock.
6,PlayStation 5 – The Best Hidden Features and E...,age 1 of 1,2.0,1,Not Available
7,Sony DualSense Wireless Controller for PlaySta...,5899,4.4,"1,447",In stock.
8,New World For PS5 Faceplate with Cooling Vent ...,2449,4.0,1,In stock.
9,Marvel's Midnight Suns | Enhanced Edition | Pl...,3899,NA,0,In stock.
